In [3]:
# Import Libraries
import pandas as pd
import numpy as np
from statsmodels.stats.anova import AnovaRM
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests

In [4]:
# Helper Functions
def partial_eta_squared(aov):
    """Calculate partial eta-squared for ANOVA results."""
    aov = aov.copy()
    aov['pes'] = aov['F Value'] * aov['Den DF'] / (
        aov['F Value'] * aov['Den DF'] + aov['Den DF']
    )
    return aov

def analyze_roi_data(data, experiment='exp1'):
    """Analyze ROI data with multiple comparison correction."""
    results = {}
    all_tests = []
    
    if experiment == 'exp1':
        conditions = ['object', 'scramble']
        rois = ['LO', 'pIPS']
    else:
        conditions = ['tool', 'nontool']
        rois = ['PFS', 'aIPS', 'LO', 'pIPS']
        
    for roi in rois:
        for hemi in ['left', 'right']:
            roi_data = data[(data['roi'] == roi) & (data['hemi'] == hemi)]
            cond1_data = roi_data[roi_data['cond'] == conditions[0]]['mean_act'].values
            cond2_data = roi_data[roi_data['cond'] == conditions[1]]['mean_act'].values
            
            # Calculate statistics
            t_stat, p_val = ttest_rel(cond1_data, cond2_data)
            mean_diff = np.mean(cond1_data - cond2_data)
            se_diff = np.std(cond1_data - cond2_data) / np.sqrt(len(cond1_data))
            d = mean_diff / np.sqrt((np.std(cond1_data)**2 + np.std(cond2_data)**2) / 2)
            
            # Store results
            all_tests.append({
                'roi': f'{hemi}_{roi}',
                'p_value': p_val,
                't_stat': t_stat,
                'mean_diff': mean_diff,
                'se': se_diff,
                'cohens_d': d,
                'n': len(cond1_data),
                f'mean_{conditions[0]}': np.mean(cond1_data),
                f'mean_{conditions[1]}': np.mean(cond2_data)
            })
    
    # Apply Holm-Bonferroni correction
    p_values = [test['p_value'] for test in all_tests]
    _, p_corrected, _, _ = multipletests(p_values, method='holm')
    
    # Update results with corrected p-values
    for test, p_corr in zip(all_tests, p_corrected):
        results[test['roi']] = test
        results[test['roi']]['p_corrected'] = p_corr
    
    return results

def run_rmanova(data, factors):
    """Run repeated measures ANOVA with effect sizes."""
    aov = AnovaRM(data=data, 
                  depvar='mean_act',
                  subject='sub',
                  within=factors).fit()
    
    aov_table = partial_eta_squared(aov.anova_table)
    return aov_table

In [5]:
# Load Data
# Experiment 1
exp1_results = pd.read_csv("results/selectivity/selectivity_summary.csv")
filtered_exp1 = exp1_results[(exp1_results['group'] == 'control') & 
                            (exp1_results['roi'].isin(['LO', 'pIPS'])) &
                            (exp1_results['sub'] != 'sub-084')]

# Experiment 2
exp2_results = pd.read_csv("/user_data/csimmon2/git_repos/ptoc/results/tools/selectivity/selectivity_summarytoolloc.csv")
filtered_exp2 = exp2_results[exp2_results['sub'].str.contains('spaceloc')]

In [6]:
# Experiment 1 Analysis
# Full ANOVA
exp1_anova = run_rmanova(filtered_exp1, ['roi', 'hemi', 'cond'])
print("Experiment 1 ANOVA Results:")
print(exp1_anova)

# ROI Analysis
exp1_roi_results = analyze_roi_data(filtered_exp1, 'exp1')
print("\nExperiment 1 ROI Results:")
for roi, stats in exp1_roi_results.items():
    print(f"\n{roi}:")
    print(f"t({stats['n']-1}) = {stats['t_stat']:.3f}, p = {stats['p_value']:.4f}, p_corr = {stats['p_corrected']:.4f}")
    print(f"Cohen's d = {stats['cohens_d']:.3f}")
    print(f"Mean difference = {stats['mean_diff']:.3f} ± {stats['se']:.3f}")

Experiment 1 ANOVA Results:
                  F Value  Num DF  Den DF        Pr > F       pes
roi             39.763898     1.0    17.0  7.897722e-06  0.975468
hemi            10.301246     1.0    17.0  5.140748e-03  0.911514
cond            55.387486     1.0    17.0  9.623988e-07  0.982266
roi:hemi         2.803147     1.0    17.0  1.123766e-01  0.737060
roi:cond       108.745714     1.0    17.0  8.346158e-09  0.990888
hemi:cond        2.746805     1.0    17.0  1.157867e-01  0.733106
roi:hemi:cond    0.056390     1.0    17.0  8.151334e-01  0.053380

Experiment 1 ROI Results:

left_LO:
t(17) = 8.141, p = 0.0000, p_corr = 0.0000
Cohen's d = 2.474
Mean difference = 2.526 ± 0.301

right_LO:
t(17) = 8.303, p = 0.0000, p_corr = 0.0000
Cohen's d = 2.689
Mean difference = 2.314 ± 0.271

left_pIPS:
t(17) = 3.815, p = 0.0014, p_corr = 0.0028
Cohen's d = 1.233
Mean difference = 0.851 ± 0.217

right_pIPS:
t(17) = 2.760, p = 0.0134, p_corr = 0.0134
Cohen's d = 0.947
Mean difference = 0.549 ± 0.193

In [8]:
# Experiment 2 Analysis
# Full ANOVA
exp2_anova = run_rmanova(filtered_exp2, ['roi', 'hemi', 'cond'])
print("Experiment 2 ANOVA Results:")
print(exp2_anova)

# Separate ANOVAs for dorsal and ventral pathways
dorsal_data = filtered_exp2[filtered_exp2['roi'].isin(['aIPS', 'pIPS'])]
ventral_data = filtered_exp2[filtered_exp2['roi'].isin(['PFS', 'LO'])]

dorsal_anova = run_rmanova(dorsal_data, ['roi', 'hemi', 'cond'])
ventral_anova = run_rmanova(ventral_data, ['roi', 'hemi', 'cond'])

# ROI Analysis
exp2_roi_results = analyze_roi_data(filtered_exp2, 'exp2')
print("\nExperiment 2 ROI Results:")
for roi, stats in exp2_roi_results.items():
    print(f"\n{roi}:")
    print(f"t({stats['n']-1}) = {stats['t_stat']:.3f}, p = {stats['p_value']:.4f}, p_corr = {stats['p_corrected']:.4f}")
    print(f"Cohen's d = {stats['cohens_d']:.3f}")
    print(f"Mean difference = {stats['mean_diff']:.3f} ± {stats['se']:.3f}")
    
# Dorsal pathway ANOVA
dorsal_data = filtered_exp2[filtered_exp2['roi'].isin(['aIPS', 'pIPS'])]
dorsal_anova = run_rmanova(dorsal_data, ['roi', 'hemi', 'cond'])
print("Dorsal Pathway ANOVA Results:")
print(dorsal_anova)

# Ventral pathway ANOVA
ventral_data = filtered_exp2[filtered_exp2['roi'].isin(['PFS', 'LO'])]
ventral_anova = run_rmanova(ventral_data, ['roi', 'hemi', 'cond'])
print("Ventral Pathway ANOVA Results:")
print(ventral_anova)

Experiment 2 ANOVA Results:
                 F Value  Num DF  Den DF        Pr > F       pes
roi            69.788835     3.0    51.0  4.595074e-18  0.985873
hemi           25.586962     1.0    17.0  9.700100e-05  0.962388
cond            1.697378     1.0    17.0  2.100038e-01  0.629270
roi:hemi        1.617712     3.0    51.0  1.967325e-01  0.617987
roi:cond       23.018004     3.0    51.0  1.468259e-09  0.958365
hemi:cond      15.531839     1.0    17.0  1.053597e-03  0.939511
roi:hemi:cond   1.616753     3.0    51.0  1.969531e-01  0.617847

Experiment 2 ROI Results:

left_PFS:
t(17) = -2.917, p = 0.0096, p_corr = 0.0436
Cohen's d = -0.374
Mean difference = -0.230 ± 0.077

right_PFS:
t(17) = -2.545, p = 0.0209, p_corr = 0.0628
Cohen's d = -0.437
Mean difference = -0.293 ± 0.112

left_aIPS:
t(17) = 5.407, p = 0.0000, p_corr = 0.0004
Cohen's d = 0.844
Mean difference = 0.344 ± 0.062

right_aIPS:
t(17) = 3.430, p = 0.0032, p_corr = 0.0191
Cohen's d = 0.729
Mean difference = 0.225 ± 0.064

In [ ]:
# FC

In [ ]:
# PPI

In [ ]:
# Subtraction

In [ ]:
# GCA

In [ ]:
# Network